In [1]:
import numpy as np
W=np.arange(21).reshape(7,3)
W


array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17],
       [18, 19, 20]])

In [2]:
# 특정 행 추출하기
print(W[2])
print(W[6])

[6 7 8]
[18 19 20]


In [3]:
# 여러개의 행 추출하기
idx=np.array([1,0,3,0])
W[idx]

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

In [4]:
class Embedding:
    def __init__(self,W):
        self.params=[W]
        self.grads=[np.zeros_like(W)]
        self.idx=None

    def forward(self,idx):
        W,=self.params
        self.idx=idx
        out=W[idx]
        return out

In [5]:
class Embeddingdot:
    def __init__(self,W):
        self.embeded=Embedding(W)
        self.params=self.embeded.params
        self.grads=self.embeded.grads
        self.cache=None

    def forward(self,h,idx):
        target_W=self.embeded.forward(idx)
        out=np.sum(target_W*h,axis=1)

        self.cache=(h, target_W)
        return out
    
    def backward(self,dout):
        h, target_W=self.cache
        dout=dout.reshape(dout.shape[0],1)
        dtarget_W=dout*h
        self.embeded.backward(dtarget_W)
        dh=dout*target_W
        return dh

In [6]:
# 확률분포에 따라 샘플링 하기.

import numpy as np

# 0에서 9까지의 숫자 중 하나를 무작위로 샘플링
print(np.random.choice(10))

print(np. random.choice(10))

# words에서 하나만 무작위로 샘플링
words=['you', 'say', 'goodbye', 'i', 'hello', '.']
print(np.random.choice(words))

# 5개만 무작위로 샘플링
print(np.random.choice(words,size=5))

# 5개만 무작위로 샘플링(중복 없음)
print(np.random.choice(words,size=5,replace=False))

#확률 분포에 따라 샘플링
p=[0.5, 0.1, 0.05, 0.2, 0.05, 0.1]
print(np.random.choice(words, p=p))

0
6
.
['i' 'i' '.' 'goodbye' 'i']
['hello' 'say' 'i' '.' 'goodbye']
.


In [7]:
# 확률에 0.75제곱을 곱해준다.

p=[0.7, 0.29, 0.01]
new_p=np.power(p,0.75)
new_p/=np.sum(new_p)
print(new_p)

#낮은 확률의 단어가 조금 더 쉽게 샘플링 될 수 있도록 구제 조치를 취했다.

[0.64196878 0.33150408 0.02652714]


In [8]:
import numpy as np

import sys
import os

# 현재 디렉토리를 sys.path에 추가
sys.path.append(os.getcwd())

from negative_sampling_layer import UnigramSampler

corpus=np.array([0,1,2,3,4,1,2,3])
power=0.75
sample_size=2

sampler=UnigramSampler(corpus,power,sample_size)
target=np.array([1,3,0])
negative_sample=sampler.get_negative_sample(target)
print(negative_sample)

[[4 0]
 [1 2]
 [4 2]]


In [9]:
import sys
sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding, SigmoidWithLoss
import collections
from negative_sampling_layer import EmbeddingDot
# chap04/negative_sampling_layer.py
class NegativeSamplingLoss:
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        self.sample_size = sample_size 
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)]  #sample_size+1은 부정적 예의 개수가 sample_size개, 긍정적 예의 개수가 1개개
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)] #정확히는 loss_layers[0]과 embeded_dot_layers[0]이 긍정적 예를 다루는 계층이다.
        
        self.params, self.grads = [], []
        for layer in self.embed_dot_layers:                 
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, h, target):
        batch_size = target.shape[0]                                    #한꺼번에 처리하는 정답의 개수
        negative_sample = self.sampler.get_negative_sample(target)      #부정적 예를 받는다.
        
        # 긍정적 예 순전파
        score = self.embed_dot_layers[0].forward(h, target)             
        correct_label = np.ones(batch_size, dtype=np.int32)             
        loss = self.loss_layers[0].forward(score, correct_label)        
        
        # 부정적 예 순전파
        negative_label = np.zeros(batch_size, dtype=np.int32)
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i]                         #(열 추출을 한다.)
            score = self.embed_dot_layers[1 + i].forward(h, negative_target)              
            loss += self.loss_layers[1 + i].forward(score, negative_label)
            
        return loss
        #forward의 연산은 열끼리 연산을 한다. 책에서 target=[1,3,0]일 때 negative_sample=[[0,3],[1,2],[2,3]]인데 이걸
        # [1 3 0]
        # [0 1 2]
        # [3 2 3]   으로 열끼리 봐라.

    #(부정적 예의 정답 레이블은 0이다.)
    #(긍정적 예의 정답 레이블은 1이다.)
    def backward(self, dout=1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)
        
        return dh

In [10]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import Embedding
from negative_sampling_layer import NegativeSamplingLoss

class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H=vocab_size, hidden_size

        #가중치 초기화
        W_in=0.01*np.random.randn(V,H).astype('f')
        W_out=0.01*np.random.randn(V,H).astype('f')

        #계층 생성
        self.in_layers=[]
        for i in range(2*window_size):              #그냥 window_size*2만큼 layer를 생성하기 위해 이렇게 한거임.
            layer=Embedding(W_in)
            self.in_layers.append(layer)
        self.ns_loss=NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)

        #모든 가중치와 기울기를 배열에 모은다.
        layers=self.in_layers+[self.ns_loss]            #리스트+리스트 -> 리스트 안에 클래스들이 나열됨.
        self.params, self.grads=[],[]
        for layer in layers:
            self.params+=layer.params
            self.grads+=layer.grads

        #인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs=W_in

    def forward(self, contexts, target):                #여기서는 SimpleCBOW와 달리 context와 target을 입력 받을 때 단어 ID를 받는다.(원핫 인코딩으로 된 것을 받지는 않음.)
        h=0
        for i, layer in enumerate(self.in_layers):
            h+=layer.forward(contexts[:,i])
        h*=1/len(self.in_layers)
        loss=self.ns_loss.forward(h,target)
        return loss
    
    def backward(self, dout=1):
        dout=self.ns_loss.backward(dout)
        dout*=1/len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None

In [11]:
# chap04/train.py
import sys,os
sys.path.append(os.pardir)
import numpy as np
from common import config
# GPU에서 실행하려면 아래 주석을 해제하세요(CuPy 필요).
# ===============================================
config.GPU = True
# ===============================================
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from cbow import CBOW
from skip_gram import SkipGram
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb


# 하이퍼파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)
# 모델 등 생성
#model = SkipGram(vocab_size, hidden_size, window_size, corpus)
model = CBOW(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)
# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size, eval_interval=2000)
trainer.plot()

word_vecs=model.word_vecs
if config.GPU:
    word_vecs=to_cpu(word_vecs)
params={}
params['word_vecs']=word_vecs.astype(np.float16)
params['word_to_id']=word_to_id
params['id_to_word']=id_to_word
pkl_file='cbow_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)

TypeError: Implicit conversion to a NumPy array is not allowed. Please use `.get()` to construct a NumPy array explicitly.

In [ ]:
import sys
sys.path.append('..')
import pickle
from common.util import most_similar, analog
pkl_file = './cbow_params.pkl'
with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
word_vecs = params['word_vecs']
word_to_id = params['word_to_id']
id_to_word = params['id_to_word']
# 가장 비슷한(most similar) 단어 뽑기
querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)

ImportError: cannot import name 'analog' from 'common.util' (c:\git_experiment\RWL_Intern\dh_workspace\밑바닥부터 시작하는 딥러닝2\common\util.py)

In [ ]:
exit()

: 